# Exploratory Data Analysis for Web Development Project

# Introduction

We're starting this project by doing some EDA on the `vehicles_us.csv` file. 

# Process Proper

## Import Libraries

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats as st
import numpy as np

## Load the File

In [2]:
try:
    data = pd.read_csv('https://raw.githubusercontent.com/vekim91/webdev_proj/main/vehicles_us.csv')
except:
    data = pd.read_csv('../vehicles_us.csv')
    


## EDA

In [3]:
# Let's take a quick glance at the dataset
data.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [4]:
# Now, let's check the columns, data types, number of entries, and number of non-null values per row
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [5]:
# Just isolating the null values per row for easier visuals
data.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

In [6]:
# Now let's check for any dupliacated rows as a whole. Since we dont have car ids, we need to take them as a whole row if there are any dupes.
data.duplicated().sum()

0

In [7]:
# And just some general stats on our data.
data.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


### Initial Impressions

There are some questions and goals that we can see once we've had a quick glance at our dataset. Here are some after looking at the information we gathered from our dataset:
1. How do we deal with the missing values in the model_year, cylinders, odometer, and paint_color columns in our data?
2. What are the best visualizations to use to better digest the data and communicate with our audience?
3. If my inteded audience is a consumer wanting to go into a buy-and-sell business, what advise should i give them

### Fix the Data

Let's start with the missing values is the model_year column. Let's tackle it this way - let's match the year to a similar make and model and use other indicative variables for value like the odometer.

But wait, are we even sure that the odometer is correlated with the price? Let's make these adjustments!

In [8]:
# Make and model of car is combined in 1 column, let's split that up.
data[['make','model']] = data['model'].str.split(' ',n=1,expand=True)
data.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
11346,9299,2015.0,cruze,good,4.0,gas,NaN,automatic,other,blue,NaN,2018-11-05,20,chevrolet
8357,22995,2014.0,f-150,excellent,8.0,gas,71808.0,automatic,truck,NaN,1.0,2018-08-31,7,ford
4507,29999,2015.0,f250,excellent,8.0,diesel,150089.0,automatic,pickup,NaN,1.0,2019-03-30,14,ford
34164,4700,2004.0,f150,good,8.0,gas,177000.0,automatic,truck,black,NaN,2019-04-10,15,ford
5361,4999,2006.0,rav4,like new,4.0,gas,166300.0,automatic,SUV,white,1.0,2018-10-03,24,toyota
27833,2900,2009.0,civic,excellent,4.0,gas,190222.0,automatic,sedan,silver,NaN,2018-12-17,0,honda
44083,28699,2013.0,wrangler unlimited,like new,6.0,gas,34000.0,automatic,SUV,white,1.0,2018-08-12,0,jeep
17124,25995,2016.0,sierra 1500,good,8.0,gas,115806.0,automatic,pickup,red,1.0,2018-05-12,50,gmc
6647,5500,2007.0,odyssey,good,6.0,gas,111347.0,automatic,mini-van,blue,NaN,2018-11-03,13,honda
13243,1200,2001.0,cr-v,fair,4.0,gas,153000.0,automatic,SUV,NaN,1.0,2018-12-01,73,honda


In [9]:
# Now let's check for correlation between odometer and price
price_odo_corr = data['price'].corr(data['odometer'])
price_odo_corr

-0.4202989623372084

In [11]:
corr_matrix = data.corr(numeric_only=True)
corr_matrix

,price,model_year,cylinders,odometer,is_4wd,days_listed
price,1.000000,0.428656,0.296627,-0.420299,NaN,-0.000682
model_year,0.428656,1.000000,-0.148040,-0.473948,NaN,-0.005703
cylinders,0.296627,-0.148040,1.000000,0.090160,NaN,0.005517
odometer,-0.420299,-0.473948,0.090160,1.000000,NaN,0.003804
is_4wd,NaN,NaN,NaN,NaN,NaN,NaN
days_listed,-0.000682,-0.005703,0.005517,0.003804,NaN,1.000000
